# Chapter 7 — Functions as First-Class Objects

## Treating a Function Like an Object

#### Example 7-1. Create and test a function, then read its `__doc__` and check its type

In [1]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)


factorial(42)

1405006117752879898543142606244511569936384000000000

In [2]:
factorial.__doc__

'returns n!'

In [3]:
type(factorial)

function

In [4]:
help(factorial)

Help on function factorial in module __main__:

factorial(n)
    returns n!



#### Example 7-2. Use `factorial` through a different name, and pass `factorial` as an argument

In [5]:
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
map(factorial, range(11))

In [8]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions

#### Example 7-3. Sorting a list of words by length

In [9]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

#### Example 7-4. Sorting a list of words by their reversed spelling

In [10]:
def reverse(word):
    return word[::-1]


reverse('testing')

'gnitset'

In [11]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Modern Replacements for map, filter, and reduce

#### Example 7-5. Lists of factorials produced with `map` and `filter` compared to alternatives coded as list comprehensions

In [12]:
list(map(factorial, range(6)))

[1, 1, 2, 6, 24, 120]

In [13]:
[factorial(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [14]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [15]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

#### Example 7-6. Sum of integers up to 99 performed with `reduce` and `sum`

In [16]:
from functools import reduce
from operator import add

reduce(add, range(100))

4950

In [17]:
sum(range(100))

4950

## Anonymous Functions

#### Example 7-7. Sorting a list of words by their reversed spelling using `lambda`

In [18]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Nine Flavors of Callable Objects

In [19]:
abs, str, 'Ni!'

(<function abs(x, /)>, str, 'Ni!')

In [20]:
[callable(obj) for obj in (abs, str, 'Ni!')]

[True, True, False]

## User-Defined Callable Types

#### Example 7-8. [`bingocall.py`](bingocall.py): A `BingoCage` does one thing: picks items from a shuffled list

In [21]:
import random


class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [22]:
bingo = BingoCage(range(3))
bingo.pick()

0

In [23]:
bingo()

1

In [24]:
callable(bingo)

True

## From Positional to Keyword-Only Parameters

#### Example 7-9. [`tagger.py`](tagger.py): `tag` generates HTML elements; a keyword-only argument `class_` is used to pass "class" attributes as a workaround because `class` is a keyword in Python

In [25]:
def tag(name, *content, class_=None, **attrs):
    """Generate one of more HTML tags"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value
                  in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>'
                    for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'

#### Example 7-10. Some of the many ways of calling the `tag` function from **Example 7-9**

In [26]:
tag('br')

'<br />'

In [27]:
tag('p', 'hello')

'<p>hello</p>'

In [28]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [29]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [30]:
print(tag('p', 'hello', 'world', class_='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [31]:
tag(content='testing', name='img')

'<img content="testing" />'

In [32]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset,jpg', 'class': 'framed'}

In [33]:
tag(**my_tag)

'<img class="framed" src="sunset,jpg" title="Sunset Boulevard" />'

## Packages for Functional Programming

### The `operator` module

#### Example 7-11. Factorial implemented with `reduce` and an anonymous function

In [34]:
from functools import reduce


def factorial(n):
    return reduce(lambda a, b: a * b, range(1, n + 1))

#### Example 7-12. Factorial implemented with `reduce` and `operator.mul`

In [35]:
from functools import reduce
from operator import mul


def factorial(n):
    return reduce(mul, range(1, n + 1))

#### Example 7-13. Demo of `itemgetter` to sort a list of tuples

In [36]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [37]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'São Paulo')


#### Example 7-14. Demo of `attrgetter` to process a previously defined list of `namedtuple` called `metro_data`

In [38]:
from collections import namedtuple

LatLon = namedtuple('LatLon', 'lat lon')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLon(lat, lon))
               for name, cc, pop, (lat, lon) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLon(lat=35.689722, lon=139.691667))

In [39]:
metro_areas[0].coord.lat

35.689722

In [40]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('São Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [41]:
import operator

[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

#### Example 7-15. Demo of `methodcaller`: second test shows the binding of extra arguments

In [42]:
from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [43]:
hyphenate = methodcaller('replace', ' ', '-')
hyphenate(s)

'The-time-has-come'

In [44]:
str.upper(s)

'THE TIME HAS COME'

### Freezing Arguments with functools.partial

#### Example 7-16. Using `partial` to use a two-argument function where a one-argument callable is required

In [45]:
from functools import partial

triple = partial(mul, 3)
triple(7)

21

In [46]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

#### Example 7-17. Building a convenient Unicode normalizing function with `partial`

In [47]:
import unicodedata

nfc = partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [48]:
s1 == s2

False

In [49]:
nfc(s1) == nfc(s2)

True

#### Example 7-18. Demo of `partial` applied to the function `tag`

In [50]:
picture = partial(tag, 'img', class_='pic-frame')
picture(src='wumpus.jpeg')

'<img class="pic-frame" src="wumpus.jpeg" />'

In [51]:
picture

functools.partial(<function tag at 0x7fe26d5aecb0>, 'img', class_='pic-frame')

In [52]:
picture.func

<function __main__.tag(name, *content, class_=None, **attrs)>

In [53]:
picture.args

('img',)

In [54]:
picture.keywords

{'class_': 'pic-frame'}